In [101]:
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime, timedelta
import smtplib
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart

#Extracting the data from the database
def extract_from_db():
    #define today's date to pull today's jobs from the database
    date = datetime.today().strftime('%Y-%m-%d')
    #define the connection to postgres
    engine = create_engine('postgresql://****@localhost:****/****') #replace **** with user, port and database name
    #define the query
    query = '''select * 
                from jobs 
                where posting_date = '{}' '''
    #import the data
    df = pd.read_sql_query(query.format(date), 
                           con=engine)
    return(df)

#Filtering the descriptions based on my requirements (e.g. No senior roles, jobs that require python and SQL, etc)
def filter_job_descriptions(df):
    #Everthing gets lowercased
    df['description'] = df['description'].str.lower()
    #We filter for python, sql and ml as those are my interests
    python_filter = df['description'].str.contains(("python"))
    sql_filter = df['description'].str.contains(("sql"))
    ml_filter = sql_filter = df['description'].str.contains(("machine learning"))
    #Apply the filters defined above
    desc_filtered = df[(python_filter & sql_filter) | ml_filter]
    #Filter out senior roles
    title_filtered = desc_filtered[~desc_filtered['job_title'].str.contains("Manager|Senior|Lead|Principal")]
    #Final df to be sent will just contain the title, the company and the url
    final_df = title_filtered[['job_title','company','job_url']]
    return(final_df)

#Email the filtered jobs
def email_filtered_jobs(final_df):
    EMAIL_ADDRESS = #From email address
    EMAIL_PASSWORD = #Password
    date = datetime.today().strftime('%Y-%m-%d')
    
    msg = MIMEMultipart()

    msg['Subject'] = "{} found jobs".format(date)
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = #To email address

    html = """Hi Adriano,\n\n
            Find attached the jobs found with your desired requirements \n\n
            <html>
              <head></head>
              <body>
                {}
              </body>
            </html>
            """.format(final_df.to_html(index=False))

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
    server.sendmail(EMAIL_ADDRESS, '*********', msg.as_string()) #replace **** with To email address

    
#Excecute the extraction, filtering and emailing
if __name__ == '__main__':
    df = extract_from_db()
    final_df = filter_job_descriptions(df)
    email_filtered_jobs(final_df)